<a href="https://colab.research.google.com/github/descandon88/Economics-Complexity/blob/main/1_netviz_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Visualize the space as a network

In this section, we will practice how to visualize the X-spaces with tradtional network-based method.

The sections are organized according to our viz-pipeline:
- Data collection and metric calculation (load previous results)
- Extract the informative part of relation: mst + high proximity
- Layout generation: force-layout
- cluster generation: community detection
- Aesthetic mapping: desired property -> visual elements
  - Whole network
  - Portfolio




Colab already provided basic packages needed for the task, such as numpy,scipy,pandas,networkx and matplotlib. Here we install two more packages: fa2 for network layout generation and python-louvain for community detection.

In [ ]:
!pip install fa2 python-louvain

In [ ]:
# load the basic packages
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

## Data collection and metric calculation (load previous results)

In this part, we need to get a proximity matrix or dataframe as the input of following steps. If you are doing your own research on some kind of spaces, you need to clean and transform your raw data, and assess the relatedness with a similarity or distance metric.

Here we will use a pre-calculated proximity dataframe from the Atlas of Economic complexity. You can get this dataframe by following Matte's tutorial on Monday, or use the py-ecomplexity package.

In [ ]:
# load the precalculated proximity dataframe
# you could load your own saved proximity data
proxurl = 'http://intl-atlas-downloads.s3.amazonaws.com/atlas_2_16_6/hs92_proximities.csv'
proxdf = pd.read_csv(proxurl,dtype={'commoditycode_1':str,'commoditycode_2':str,'proximity':float})
proxdf.head()

Let's first examine the distribution of proximity metric, which helps us decide in extracting the informative parts.

In [ ]:
proxdf.proximity.plot.hist(bins=50)

We observed that proximity is right-skewed. There are a few observations in the right tail with high proximity, which is good: we can cover the informative part without introducing many links.

Load the names of hs code so that we could use for annotation and analysis.

In [ ]:
hsurl = 'http://intl-atlas-downloads.s3.amazonaws.com/17.0/hs_product.zip'
hsdf = pd.read_csv(hsurl,compression='zip',dtype='str')
hsdf.head()

Let's only keep the product names used in the calculation of proximity:

In [ ]:
hsdf.hs_product_code.nunique(),proxdf.commoditycode_1.nunique()

In [ ]:
nodedf = hsdf[hsdf.hs_product_code.isin(proxdf.commoditycode_1)][['hs_product_code','hs_product_name_short_en']].sort_values('hs_product_code').reset_index(drop=True)
nodedf.head()

### (Optional) visualize the raw proximity matrix

If your data is a proximity dataframe, you could transform it into a proxmity matrix using the pandas "pivot" function. The matrix format is sometimes faster for density calculation and other tasks. On the other hand, you could also get a proximity dataframe using pandas "melt" or other functions.

In [ ]:
proxmat = proxdf.pivot(index='commoditycode_1', columns='commoditycode_2',values='proximity').to_numpy()
proxmat.shape

You could visualize the proximity matrix directly using a heatmap. The natural sorting of product codes usually exhibits a block structure.

In [ ]:
sns.heatmap(proxmat,cmap='viridis',square=True)

## Create graph and extract the backbone

Given the proximity dataframe or matrix, we could convert them into networkx object. By default, it will create an undirected network. We set the "edge_attr" as **True**, so every other columns in the dataframe are loaded as edge attributes:

In [ ]:
fullgraph = nx.from_pandas_edgelist(proxdf,source='commoditycode_1',target='commoditycode_2',edge_attr=True)
fullgraph.number_of_nodes(),fullgraph.number_of_edges()

Networkx provides the function to generate a minimum/maximum spanning tree, we will generate this using proximity values as the basic skeleton, which makes sure every nodes are connected.

In [ ]:
mst = nx.maximum_spanning_tree(fullgraph,weight='proximity')
mst.number_of_edges()

# Exercise 1

Try a different threshold (other than 0.55) to create the network. Below is the previous code.


```
vizgraph = mst.copy()
vizgraph.add_edges_from([(u,v,d) for u,v,d in fullgraph.edges(data=True) if d['proximity']>0.55])
vizgraph.number_of_edges()
```



In [ ]:
## put your code below



In [ ]:
nx.density(vizgraph)

## Generate layout of the nodes

Before we jump to the visualization of full network, let's check how the mst skeleton looks.

Here we use the Kamada-Kawai algorithm to generate a layout, which generally works well for smaller networks:

In [ ]:
position = nx.kamada_kawai_layout(mst)
nx.draw(mst,pos=position,node_size=1)

The result already revealed some branches and clusters, we could use this layout as an initial position and speed up the generation of `vizgraph` layout.

In this step, you would probably want to explore different layout algorithms, tune their parameters, and plot them until you get a satisfying position. Sometimes, it may require some manual adjustments before you finalize the layout.

# Exercise 2

Use another layout algorithm, such as the `sprint_layout` in networkx, and tune the parameters to generate a new usable layout.

The available layout algorithms and their documents are here: https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout

In [ ]:
## put your code below, save the layout as "position2"


The position output is a dictionary, we could create a dataframe and merge it with the nodedf for future use.

In [ ]:
nodedf = nodedf.merge(
    pd.DataFrame.from_dict(position2,orient='index',columns=['x','y']),
    how='left',
    left_on='hs_product_code',
    right_index=True
)
nodedf.head()

## Community detection

We would like to extract more meso-scale structural information out of this network representation, one option is to extract the community structure that reveal the block structure we saw in matrix plot.

There are a number of community detection algorithms. The Louvain algorithm is one of the most widely used solution, and the python implementation is here: https://python-louvain.readthedocs.io/en/latest/api.html

In [ ]:
import community as community_louvain
partition = community_louvain.best_partition(vizgraph,weight='proximity',resolution=1,random_state=42)
max(partition.values()) + 1

The `partition` object is a dictionary that sequentially maps each nodes to the community id. We could also merge it to the nodedf dataframe for further analysis.

In [ ]:
nodedf = nodedf.merge(
    pd.DataFrame.from_dict(partition,orient='index',columns=['communityid']),
    how='left',
    left_on='hs_product_code',
    right_index=True
)
nodedf.head()

Using pandas, we could check the size of each communities:

In [ ]:
nodedf.communityid.value_counts()

For each community, we could check the included products to understand its meaning.

For example, community 6 is a cluster of garments and textile products

In [ ]:
nodedf.query('communityid == 6').sample(15)

Rearrange the rows and columns in matrix plot, we see a clearer block structure

In [ ]:
idx = np.argsort(nodedf.communityid)
proxmat2 = proxmat[:,idx][idx]
sns.heatmap(proxmat2,cmap='viridis',square=True)

## Mapping properties to aesthetic elements

Once we have fixed the position of nodes in the network, the aesthetic elements we could use are mainy the color and size of the nodes. Different shapes of the nodes are not very distinguishable with >1000 nodes, and labels are only usable to annotate few nodes/sectors/communities.

In this section, we will use a saved output from running the py-ecomplexity package on 2015 HS 4-digit trade data:

In [ ]:
df_ec = pd.read_parquet('https://github.com/complexly/summerschool_viz/raw/main/df_ec.parquet')
df_ec.head()

### Color

The color of the nodes are usually used to indicate different categories, such as 2-digit sectors or the communities we discovered above.

A meaningful color map usually require some manual design, such as using brown to represent mining activities and products. Here we would just assign a color to each commmunity from the rainbow color pallette, without further improvement.

In [ ]:
from matplotlib import colors
# use the gist_rainbow colormap
cm = plt.get_cmap('gist_rainbow')
# map each communityid to a color, the output is an rgba array for each node 
colorarray = cm([x/nodedf.communityid.nunique() for x in nodedf.communityid])
# convert the rgba array into a hex string, and store the result into the nodedf dataframe
nodedf['color'] = np.apply_along_axis(colors.to_hex, 1, colorarray)
nodedf.sample(5)

Now let's use this color to make a plot. The `node_color` parameter accepts rgba array or hex string for each nodes, which we generated above and already in the correct order.

In [ ]:
plt.figure(figsize=(14,8))
nx.draw(vizgraph,pos=position2,node_size=5,node_color=nodedf.color,edge_color='lightgrey')

### Size

# Exercise 3

Use the node size to represent the logged total trade volume of a product, which is the "log_export_value" columns in the `nodedf` dataframe.

In [ ]:
nodedf = nodedf.merge(
    df_ec.groupby('hs_product_code')['export_value'].sum(),
    how='left',
    on='hs_product_code'
)
nodedf.head()

In [ ]:
nodedf['log_export_value'] = np.log(nodedf['export_value'])
nodedf.log_export_value.describe()

In [ ]:
## put your code below: calcuate the nodesize and plot the network


### Annotation

Sometimes we would like to add some annotations to the product space visualization. Here we will add the communityid to the center of each community.

In [ ]:
df_annotation = nodedf.groupby('communityid')[['x','y']].mean().reset_index()
df_annotation.head()

In [ ]:
plt.figure(figsize=(14,8))
nx.draw(vizgraph,
        pos=position2,
        node_size=nodedf.nodesize,
        node_color=nodedf.color,
        edge_color='lightgrey',
        edgecolors='grey')
for i, point in df_annotation.iterrows():
        plt.annotate(point['communityid'],(point['x'], point['y']),fontsize=15)

### Region specific plot

# Exercise 4

Plot the product highlighting where Ukraine (location code: UKR) has comparative advantage.

In [ ]:
# put your code below
